In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)
mnist = input_data.read_data_sets("../MNIST_data/", one_hot = True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
training_epoch=10
learning_rate=0.001
batch_size=100

In [3]:
X=tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X,[-1,28,28,1])
Y=tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,16], stddev=0.01)) # 3x3x1 filter를 32개 사용 -> 출력의 두께가 32
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding='SAME') # stride의 4개원소중 가운데 두개가 실제 striding하는 값
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3,3,16,32],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7*7*32,128], stddev=0.01))
L3 = tf.reshape(L2, [-1,7*7*32])
L3 = tf.nn.relu(tf.matmul(L3,W3))
L3 = tf.nn.dropout(L3, keep_prob) # FC에 적용하는 dropout

W4 = tf.Variable(tf.random_normal([128,10]))
model = tf.matmul(L3,W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [4]:
total_batch = int(mnist.train.num_examples/batch_size)

print('Learning Start!')

sess=tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epoch):
    total_cost=0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _= sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
        total_cost+=c
        
    print('Epoch:','{:3d}'.format(epoch+1), 'Cost:','%5f'%(total_cost/total_batch))
        
print('Learning Finished!')
        

Learning Start!
Epoch:   1 Cost: 0.293891
Epoch:   2 Cost: 0.107221
Epoch:   3 Cost: 0.078092
Epoch:   4 Cost: 0.065867
Epoch:   5 Cost: 0.053675
Epoch:   6 Cost: 0.046514
Epoch:   7 Cost: 0.039414
Epoch:   8 Cost: 0.034014
Epoch:   9 Cost: 0.029981
Epoch:  10 Cost: 0.027737
Learning Finished!


In [5]:
is_correct = tf.equal(tf.argmax(Y,1),tf.argmax(model,1)) #argmax의 axis=1인 경우 행에서의 최대값을 찾음
                                                        #axis=0 열 기준 axis=1 행 기준
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32)) #tf.cast는 tf.float32의 자료형으로 casting(변환)해줌
print('acc:',(sess.run(accuracy,
                          feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1})))

acc: 0.99030006


In [17]:
import random
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(model, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

Label:  [9]
Prediction:  [9]
